In [ ]:
import pandas as pd
import numpy as np
from pandas import DataFrame
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, cross_val_score
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.preprocessing import LabelEncoder
!pip install catboost
from catboost import CatBoostClassifier

In [ ]:
train= pd.read_csv('/content/drive/MyDrive/WIDS Stanford/TrainingWiDS2021.csv/TrainingWiDS2021.csv')
test= pd.read_csv('/content/drive/MyDrive/WIDS Stanford/UnlabeledWiDS2021.csv/UnlabeledWiDS2021.csv')
val= pd.read_csv('/content/drive/MyDrive/WIDS Stanford/UnlabeledWiDS2021.csv/UnlabeledWiDS2021.csv')
print("train_shape:", train.shape)
print('test_shape:', test.shape)

train_shape: (130157, 181)
test_shape: (10234, 180)


In [ ]:
cat_feat= train.select_dtypes(include=['object'])

In [ ]:
cat_feat

,ethnicity,gender,hospital_admit_source,icu_admit_source,icu_stay_type,icu_type
0,Caucasian,M,Floor,Floor,admit,CTICU
1,Caucasian,F,Floor,Floor,admit,Med-Surg ICU
2,Caucasian,F,Emergency Department,Accident & Emergency,admit,Med-Surg ICU
3,Caucasian,F,Operating Room,Operating Room / Recovery,admit,CTICU
4,Caucasian,M,NaN,Accident & Emergency,admit,Med-Surg ICU
...,...,...,...,...,...,...
130152,Caucasian,M,Emergency Department,Accident & Emergency,admit,Cardiac ICU
130153,Caucasian,F,Direct Admit,Accident & Emergency,admit,MICU
130154,African American,M,Emergency Department,Accident & Emergency,admit,Cardiac ICU
130155,Caucasian,M,Emergency Department,Accident & Emergency,admit,Med-Surg ICU


# Cat Features

In [ ]:
cat_feats= ['ethnicity', 'gender', 'hospital_admit_source', 'icu_admit_source', 'icu_stay_type', 'icu_type']
for x in cat_feats:
  le= LabelEncoder()
  train[x]= le.fit_transform(train[x].astype(str))

In [ ]:
cat_feats= ['ethnicity', 'gender', 'hospital_admit_source', 'icu_admit_source', 'icu_stay_type', 'icu_type']
for x in cat_feats:
  le= LabelEncoder()
  test[x]= le.fit_transform(test[x].astype(str))

In [ ]:
train.drop(['Unnamed: 0', 'encounter_id', 'hospital_id'], axis = 1, inplace = True)

In [ ]:
test.drop(['Unnamed: 0', 'encounter_id', 'hospital_id'], axis = 1, inplace = True)

# Modeling

In [ ]:
X= train.drop('diabetes_mellitus', 1)
y= train['diabetes_mellitus']

In [ ]:
cb_params={
    'n_estimators': 3500,
    #'learning_rate':0.01,
    'random_seed': 3500,
    'objective':'CrossEntropy',
    'eval_metric': 'AUC',
    'max_depth': 8,
    'thread_count': -1,
}

In [ ]:
cb= CatBoostClassifier(**cb_params, verbose= False)

In [ ]:
scorevalid, scoretest= [], []
layers= StratifiedKFold(n_splits= 2, shuffle= True, random_state=999)

In [ ]:
for  i, (trainvalue, valvalue) in enumerate (layers.split(X, y)):
  train_x, val_x=  X.iloc[trainvalue], X.iloc[valvalue]
  train_y, val_y= y.iloc[trainvalue], y.iloc[valvalue]
  print('#'*50 + 'fold'+ str(i))

  cb.fit(train_x, train_y, cat_features= cat_feats)
  pred= cb.predict_proba(val_x)[:,1]
  valpred= roc_auc_score(val_y, pred)
  scorevalid.append(valpred)

  predict= cb.predict_proba(test)[:, 1]
  scoretest.append(predict)

##################################################fold0
##################################################fold1


In [ ]:
print(np.mean(scorevalid))

0.8659798818593794


In [ ]:
result= pd.DataFrame(scorevalid).T

In [ ]:
result.head()

In [ ]:
final_pred= np.mean(scoretest, axis=0)

# Submission File

In [ ]:
submission = val[['encounter_id','hospital_id']]

In [ ]:
submission['diabetes_mellitus']= final_pred
submission.drop('hospital_id', 1, inplace= True)
submission.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,encounter_id,diabetes_mellitus
0,144740,0.035113
1,141990,0.084127
2,142038,0.094958
3,138628,0.029812
4,141682,0.323680


In [ ]:
submission.to_csv("standfords.csv", index=False)

In [ ]:
print(classification_report(val_y, preds))

NameError: ignored